What now?

We create two tables with the following data:

# Table 1
- Based on basic data in `~/data/trd/crosses_info.csv`, we want to know 
    - number of loci with
        - TRD
        - of these
            - positive, negative outlier
                - pi
                - fst
                - dxy
                - LD
                - local phylogeny data
                - dn/ds
            - SVs
                - for each SV type, count of loci that overlap with these
            - repetitive elements
                - % region in rep. bp

# Table 2
- All individual TRD loci data, with per locus
    - percentile values for
        - pi
        - fst
        - dxy
        - LD
        - local phylogeny data
        - dn/ds
    - SVs, each type
        - % of loci overlapping with called SV
    - rep elements, each type
        - % locus

After creating those tables, we should also create a PDF report for each region.

---
title: "Analysis of Genome Region"
output: pdf_document
---

## Introduction

This is a report on the analysis of genome region `r region_name`.

## Histogram of Variable

```{r}
ggplot(data, aes(x = variable)) +
  geom_histogram() +
  labs(title = paste("Histogram of", variable)) +
  theme_minimal()



rmarkdown::render("path/to/your/file.Rmd", params = list(region_name = region_name))
````